In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import seaborn as sn

import warnings
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
df_main = pd.read_csv('class_data.csv', engine='python',encoding='utf-8')

In [ ]:
target_value = pd.read_csv('class_target_value.csv')

In [ ]:
df = df_main.copy()
df.shape

(1093, 4)

In [ ]:
df.tail(10)

,Index,code,comment,label
1083,1084,sealed class RerereCommand : Command\n{\npubli...,You need to set the configuration variable rer...,1
1084,1085,public sealed class ResetBranchParameters\n{\n...,Parameters for IBranchAccessor.ResetBranch ope...,1
1085,1086,public sealed class ResetCommand : Command\n{\...,Reset current HEAD to the specified state.,1
1086,1087,public sealed class ResetFilesParameters\n{\n/...,Parameters for IIndexAccessor.ResetFiles opera...,1
1087,1088,internal class Resources {\nprivate static glo...,"A strongly-typed resource class, for looking u...",1
1088,1089,internal class Resources {\nprivate static glo...,"A strongly-typed resource class, for looking u...",0
1089,1090,internal class Resources {\nprivate static glo...,"A strongly-typed resource class, for looking u...",0
1090,1091,internal class Resources {\nprivate static glo...,"A strongly-typed resource class, for looking u...",0
1091,1092,internal class Resources {\nprivate static glo...,"A strongly-typed resource class, for looking u...",0
1092,1093,internal class Resources {\nprivate static glo...,"A strongly-typed resource class, for looking u...",0


In [ ]:
target_value.head(10)

,Index,label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1


In [ ]:
df.label.value_counts()

1    1021
0      72
Name: label, dtype: int64

In [ ]:
# df.loc[:,'label'] = df.label.map({'NOT_COHERENT':0, 'COHERENT':1})

In [ ]:
# target_value.loc[:,'Label'] = target_value.Label.map({'NOT_COHERENT':0, 'COHERENT':1})

In [ ]:
target_value.head(5)

,Index,label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [ ]:
df.head(5)

,Index,code,comment,label
0,1,public static class AddC\n{\n/// <summary>\n//...,Adds a constant value to each sample of a signal.,1
1,2,public static class AddProduct\n{ /// <summary...,Pixel by pixel addition of product of pixels f...,1
2,3,public static class AddProductC\n{\n///// <sum...,Adds product of a constant and each sample of ...,1
3,4,public static class AddProductSignal\n{ /// <s...,Adds sample by sample product of two signals t...,1
4,5,public static class AddSignal\n{\t/// <summary...,Sample by sample addition of two signals.,1


In [ ]:
!pip install stopwordsiso
import stopwordsiso as stopwords

stop_words = stopwords.stopwords("en")
# print(stop_words)

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 73 kB 1.7 MB/s 


[nltk_data] Downloading package wordnet to /root/nltk_data...


# **Function Declaration**

**Data Cleaning**

In [ ]:
stop_words.add('')

In [ ]:
def cleaning(data):
  data = data.str.replace('[^\w\s]','')
  data = data.replace('/+','', regex=True)
  data = data.replace('\n','', regex=True)
  data = data.replace('\t','', regex=True)
  data = data.replace('_','', regex=True)
  data = data.replace('public','', regex=True)
  data = data.replace('void','', regex=True)
  data = data.replace('static','', regex=True)
  data = data.replace('class','', regex=True)
  data = data.str.split(' ')
  data = data.apply(lambda x: [item for item in x if item not in stop_words])
  #data = data.apply(lambda x: [stemmer.stem(y) for y in x])
  data = data.apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
  return data

In [ ]:
# x=cleaning(df)
#x.head(5)

**CountVectorizer for train data**

In [ ]:
def CV_train(text):
  count_vectorizer = CountVectorizer(stop_words='english', max_features=40000)
  document_term_matrix = count_vectorizer.fit_transform(text)
  return count_vectorizer, document_term_matrix

**CountVectorizer for test data**

In [ ]:
def CV_test(count_vectorizer, text):
  document_term_matrix = count_vectorizer.transform(text)
  return document_term_matrix

**LDA( Topic Modeling Technique) for train data**

In [ ]:
def LDA_train(n_topics, document_term_matrix):
  lda_model = LatentDirichletAllocation(n_components=n_topics, learning_method='online', random_state=0, verbose=0)
  lda_topic_matrix = lda_model.fit_transform(document_term_matrix)
  return lda_model, lda_topic_matrix

**LDA( Topic Modeling Technique) for test data**

In [ ]:
def LDA_test(lda_model, n_topics, document_term_matrix):
  lda_topic_matrix = lda_model.transform(document_term_matrix)
  return lda_topic_matrix

**Feature Concatination**

In [ ]:
def get_concatenated_features(comment_vec_matrix, code_vec_matrix):
  Concatenated_Features_array= []
  for x in range(len(code_vec_matrix)):
    row_array = []
    for y in range(len(code_vec_matrix[0])):
      row_array.append(code_vec_matrix[x][y])
      row_array.append(comment_vec_matrix[x][y])
    Concatenated_Features_array.append(row_array)
  return Concatenated_Features_array

**Random Forest model fit on train data**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def RF_Inference_train(concat_feature, target_value):
  model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
  model.fit(concat_feature, target_value)
  probablities = model.predict_proba(concat_feature)
  return model, probablities

**Random Forest model transform on test data using probability**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def RF_Inference_test(model, concat_feature):
  probablities = model.predict_proba(concat_feature)
  return probablities

# Function Calls after using stratified kfold

In [ ]:
accuracies = []
roc_auc =[]
precision = []
recall = []

X = df.to_numpy()
y = target_value['label'].to_numpy()

skf = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
#skf = StratifiedKFold(n_splits=2, random_state=1, shuffle=True)
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
    train_X, test_X = pd.DataFrame(X[train_index], columns=df.columns), pd.DataFrame(X[test_index], columns=df.columns)
    train_y, test_y = pd.DataFrame(y[train_index], columns=[' Label']), pd.DataFrame(y[test_index], columns=[' Label'])

    train_X['code'] = train_X['code'].astype(str)
    train_X['comment'] = train_X['comment'].astype(str)

    test_X['code'] = test_X['code'].astype(str)
    test_X['comment'] = test_X['comment'].astype(str)

    #function calls

    train_X['code'] = cleaning(train_X['code']).astype(str)
    train_X['comment'] = cleaning(train_X['comment']).astype(str)


    test_X['code'] = cleaning(test_X['code']).astype(str)
    test_X['comment'] = cleaning(test_X['comment']).astype(str)

    #CV for train data
    comment_cv_model, comment_matrix_train = CV_train(train_X["comment"])
    code_cv_model, code_matrix_train = CV_train(train_X["code"])

    #CV for test data
    comment_matrix_test = CV_test(comment_cv_model, test_X["comment"])
    code_matrix_test = CV_test(code_cv_model, test_X["code"])

    #for LDA and concatenated features
    comment_lda_models_train = []
    code_lda_models_train = []

    rf_models_train = []

    proba_list_train = []
    for i in range(5, 26):
        comment_lda_model, comment_LDA_train = LDA_train(i, comment_matrix_train)
        comment_lda_models_train.append(comment_lda_model)
        code_lda_model, code_LDA_train = LDA_train(i, code_matrix_train)
        code_lda_models_train.append(code_lda_model)
        features = get_concatenated_features(comment_LDA_train, code_LDA_train)
        rf_model, proba = RF_Inference_train(features, train_y)
        rf_models_train.append(rf_model)
        proba_list_train.append(proba)
    
    proba_list_test = []
    for i in range(5, 26):
        comment_LDA_test = LDA_test(comment_lda_models_train[i-5], i, comment_matrix_test)
        code_LDA_test = LDA_test(code_lda_models_train[i-5], i, code_matrix_test)
        features = get_concatenated_features(comment_LDA_test, code_LDA_test)
        proba_test = RF_Inference_test(rf_models_train[i-5], features)
        proba_list_test.append(proba_test)

    rf5 = proba_list_train[0][:,1]
    rf6 = proba_list_train[1][:,1]
    rf7 = proba_list_train[2][:,1]
    rf8 = proba_list_train[3][:,1]
    rf9 = proba_list_train[4][:,1]
    rf10 = proba_list_train[5][:,1]
    rf11 = proba_list_train[6][:,1]
    rf12 = proba_list_train[7][:,1]
    rf13 = proba_list_train[8][:,1]
    rf14 = proba_list_train[9][:,1]
    rf15 = proba_list_train[10][:,1]
    rf16 = proba_list_train[11][:,1]
    rf17 = proba_list_train[12][:,1]
    rf18 = proba_list_train[13][:,1]
    rf19 = proba_list_train[14][:,1]
    rf20 = proba_list_train[15][:,1]
    rf21 = proba_list_train[16][:,1]
    rf22 = proba_list_train[17][:,1]
    rf23 = proba_list_train[18][:,1]
    rf24 = proba_list_train[19][:,1]
    rf25 = proba_list_train[20][:,1]

    #for train data
    df_rf_train = pd.DataFrame(list(zip(rf5, rf6, rf7, rf8, rf9, rf10, rf11, rf12, \
                                        rf13, rf14, rf15, rf16, rf17, rf18, rf19, \
                                        rf20, rf21, rf22, rf23, rf24, rf25\
                                        )), columns =['rf5', 'rf6', 'rf7', 'rf8', \
                                                      'rf9', 'rf10', 'rf11', 'rf12', \
                                                      'rf13', 'rf14', 'rf15', 'rf16', \
                                                      'rf17', 'rf18', 'rf19', 'rf20', \
                                                      'rf21', 'rf22', 'rf23', 'rf24', 'rf25'])
    
    
    # all parameters not specified are set to their defaults
    logisticRegr = LogisticRegression()
    logisticRegr.fit(df_rf_train, train_y)

    #for test data
    rf5 = proba_list_test[0][:,1]
    rf6 = proba_list_test[1][:,1]
    rf7 = proba_list_test[2][:,1]
    rf8 = proba_list_test[3][:,1]
    rf9 = proba_list_test[4][:,1]
    rf10 = proba_list_test[5][:,1]
    rf11 = proba_list_test[6][:,1]
    rf12 = proba_list_test[7][:,1]
    rf13 = proba_list_test[8][:,1]
    rf14 = proba_list_test[9][:,1]
    rf15 = proba_list_test[10][:,1]
    rf16 = proba_list_test[11][:,1]
    rf17 = proba_list_test[12][:,1]
    rf18 = proba_list_test[13][:,1]
    rf19 = proba_list_test[14][:,1]
    rf20 = proba_list_test[15][:,1]
    rf21 = proba_list_test[16][:,1]
    rf22 = proba_list_test[17][:,1]
    rf23 = proba_list_test[18][:,1]
    rf24 = proba_list_test[19][:,1]
    rf25 = proba_list_test[20][:,1]

    df_rf_test = pd.DataFrame(list(zip(rf5, rf6, rf7, rf8, rf9, rf10, rf11, rf12, rf13, rf14, rf15, rf16,\
                         rf17, rf18, rf19, rf20, rf21, rf22, rf23, rf24, rf25)),
               columns =['rf5', 'rf6', 'rf7', 'rf8', 'rf9', 'rf10', 'rf11', 'rf12', 'rf13', 'rf14', 'rf15', 'rf16',\
                         'rf17', 'rf18', 'rf19', 'rf20', 'rf21', 'rf22', 'rf23', 'rf24', 'rf25'])
    print(test_y)
    #roc_auc_y = test_y.loc[:,'label'] = test_y.label.map({'NOT_COHERENT':0, 'COHERENT':1})
    accuracies.append(accuracy_score(test_y, logisticRegr.predict(df_rf_test)))
    roc_auc.append(roc_auc_score(test_y, logisticRegr.predict(df_rf_test)))
    precision.append(precision_score(test_y, logisticRegr.predict(df_rf_test)))
    recall.append(recall_score(test_y, logisticRegr.predict(df_rf_test)))

    print(classification_report(test_y,logisticRegr.predict(df_rf_test)))
    print(f'roc_auc score: {roc_auc_score(test_y,logisticRegr.predict(df_rf_test))}')
    print(f'precision: {precision_score(test_y, logisticRegr.predict(df_rf_test))}')
    print(f'recall: {recall_score(test_y, logisticRegr.predict(df_rf_test))}')

TRAIN: (983,) TEST: (110,)
      Label
0         1
1         1
2         1
3         1
4         1
..      ...
105       1
106       1
107       1
108       1
109       1

[110 rows x 1 columns]
              precision    recall  f1-score   support

           0       0.40      0.57      0.47         7
           1       0.97      0.94      0.96       103

    accuracy                           0.92       110
   macro avg       0.69      0.76      0.71       110
weighted avg       0.93      0.92      0.92       110

roc_auc score: 0.7565880721220527
precision: 0.97
recall: 0.941747572815534
TRAIN: (983,) TEST: (110,)
      Label
0         1
1         1
2         1
3         1
4         1
..      ...
105       1
106       1
107       1
108       1
109       0

[110 rows x 1 columns]
              precision    recall  f1-score   support

           0       0.40      0.50      0.44         8
           1       0.96      0.94      0.95       102

    accuracy                           0.91

In [ ]:
print(f'accuracies:{accuracies}')

accuracies:[0.9181818181818182, 0.9090909090909091, 0.9, 0.908256880733945, 0.9541284403669725, 0.9174311926605505, 0.908256880733945, 0.9724770642201835, 0.8990825688073395, 0.926605504587156]


In [ ]:
print(f'roc_auc scores:{roc_auc}')

roc_auc scores:[0.7565880721220527, 0.7205882352941176, 0.6580882352941176, 0.7514005602240895, 0.9089635854341738, 0.7563025210084032, 0.6848739495798319, 0.9187675070028012, 0.6134453781512605, 0.6946778711484594]


In [ ]:
print(f'precision scores: {precision}')

precision scores: [0.97, 0.96, 0.9504950495049505, 0.9693877551020408, 0.98989898989899, 0.9696969696969697, 0.96, 0.9900990099009901, 0.9504950495049505, 0.9607843137254902]


In [ ]:
print(f'recall scores: {recall}')

recall scores: [0.941747572815534, 0.9411764705882353, 0.9411764705882353, 0.9313725490196079, 0.9607843137254902, 0.9411764705882353, 0.9411764705882353, 0.9803921568627451, 0.9411764705882353, 0.9607843137254902]


# 1-gram individual BLEU

In [ ]:
# from nltk.translate.bleu_score import sentence_bleu
# scores = []
# for index, rows in df.iterrows():
#     reference = [str(rows['code']).split()]
#     candidate = str(rows['comment']).split()
#     scores.append(sentence_bleu(reference, candidate))
#     print(f"Label : {rows['label']} vs BLEU SCORE: {scores}")

**To generate the CSV file of BLEU Score**

In [ ]:
# bleu_csv = pd.DataFrame()

# bleu_csv['code'] = df['code']
# bleu_csv['comment'] = df['comment']
# bleu_csv['label'] = df['label']
# bleu_csv['bleu_score'] = scores

# bleu_csv.head(10)

# bleu_csv.to_csv("bleu_score_1.csv")

#2-gram BLEU Score

In [ ]:
# n-gram individual BLEU
# from nltk.translate.bleu_score import sentence_bleu
# reference = [['this', 'is', 'a', 'test']]
# candidate = ['this', 'is', 'a', 'test']
# print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
# print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
# print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
# print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

In [ ]:
# scores = []
# for index, rows in df.iterrows():
#     reference = [str(rows['code']).split()]
#     candidate = str(rows['comment']).split()
#     scores.append(sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
#     print(f"Label : {rows['label']} vs BLEU SCORE: {scores}")

In [ ]:
# bleu_csv = pd.DataFrame()

# bleu_csv['code'] = df['code']
# bleu_csv['comment'] = df['comment']
# bleu_csv['label'] = df['label']
# bleu_csv['bleu_score'] = scores

# bleu_csv.head(10)

# bleu_csv.to_csv("bleu_score_2.csv")

#k-Alpha

In [ ]:
# !pip install simpledorff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ann_1 = pd.read_csv('annotator1.csv')
# ann_1['pair_id'] = ann_1.index
# ann_1.rename(columns = {'Label(Lima)':'label'}, inplace = True)
# ann_1['annotator_id'] = 'Lima'
# ann_1.head(5)

FileNotFoundError: ignored

In [ ]:
# ann_2 = pd.read_csv('annotator2.csv')
# ann_2['pair_id'] = ann_2.index
# ann_2.rename(columns = {'Label(Talha)':'label'}, inplace = True)
# ann_2['annotator_id'] = 'Talha'
# ann_2.head(5)

In [ ]:
# ann_3 = pd.read_csv('annotator3.csv')
# ann_3['pair_id'] = ann_3.index
# ann_3.rename(columns = {'Label(Safayat)':'label'}, inplace = True)
# ann_3['annotator_id'] = 'Safayat'
# ann_3.head(5)

In [ ]:
# merged = pd.merge(ann_1, ann_2, on=['label', 'pair_id', 'annotator_id'], how='outer')

In [ ]:
# merged_final = pd.merge(merged, ann_3, on=['label', 'pair_id', 'annotator_id'], how='outer')

In [ ]:
# import simpledorff
# k_alpha = simpledorff.calculate_krippendorffs_alpha_for_df(merged_final,experiment_col='pair_id',
#                                                  annotator_col='annotator_id',
#                                                  class_col='label')

# print(k_alpha)